In [1]:
%run ClusterAlg.py

In [2]:
from time import clock
tic = clock()
for i in xrange(1):
    test()
toc = clock()
print('total simulation time is %f'%(toc-tic))
# m = MedianHeap([1.4, 2.1, 5.0, 5.0])
# print('median, ', m.get())

start KMedians ...
start KMedians ...
start KMedians ...
start KMedians ...
start KMedians ...
start KMedians ...
start KMedians ...
start KMedians ...
start KMedians ...
start KMedians ...
cluster,  [0, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 0, 2, 2]
distToClusterCenter,  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
total simulation time is 2.828105
